# Easy-Trustflow
Easy-Trustflow的目标是让每个人用得对,用得好TEE！

## 什么是安全的TEE应用
首先，我们需要认识到，并不是把应用在TEE里跑起来就意味着安全。数据在传输、存储、计算等各方面如果设计不当，尤其是多方数据协作计算时，依然可能产生数据滥用或者甚至泄露。相信您在阅读了[Trustflow原理](./principle.md)和[快速上手](../quick_start/index.rst)之后，已经对这一点有了一定的认知。
鉴于此，我们需要对应用TEE化改造，并且对数据进行权限管控，才能称得上是一个安全的TEE应用。

## 繁重的TEE应用改造
在[快速上手第四步](../quick_start/step4.ipynb)中，我们提供了[teeapps](https://github.com/asterinas/trustflow-teeapps)这个应用框架，它能产生远程认证报告，能从Capsule Manager获取数据密钥，能对数据进行加解密并进行计算。但正如你所感觉到的，在处理复杂的任务和服务时，它显得不够灵活。
造成这个现象的一大原因是在teeapps研发之初，主流的TEE还是Intel SGX，它是一种进程级的TEE实现。受限于此，我们不得不对应用进行侵入式改造。如下图所示：

![teeapps-all-in-one](../images/teeapp_all_in_one.jpg)

虽然后续版本我们已经支持CVM（机密虚拟机）模式，但是在这种架构下应用也仅限于可以运行在CVM上，并没有享受到CVM带来的好处。在这种模式下，除了对源码侵入式修改本身带来的开发和运维成本之外，对于应用开发者而言，还需要一定程度懂TEE（比如起码了解远程认证）才能做好修改，存在额外的学习成本。另外，一个系统中通常有多个应用，这意味着开发者需要对每个应用做一次改造。如果这些应用还是不同语言编写的，那么改造工作就会变得更加繁重。
我们希望优化这个改造过程，将这些应用改造能够与应用本体解耦。

## CVM带来的机遇
TEE应用改造在CVM（机密虚拟机）场景下有了新的解法。CVM天生适合容器化，给TEE应用在云原生方面带来了新的可能性，我们可以借鉴云原生的思想，把对应用的改造变成服务模块（以sidecar形式部署），使得应用尽可能低成本甚至无需任何改造即可享受TEE带来的安全增益。

![teeapps-sidecar](../images/teeapp_sidecar.jpg)

## Easy-Trustflow架构

Easy-Trustflow的原理与[Trustflow原理](./principle.md)是描述的是相似的。区别点在于可信App从单一进程变为了一个应用本体Container和多个sidecar的形式。这将使可信App的开发将变得十分容易。开发者只需专注于开发应用本身的能力，剩下的都交给sidecar。

![easy_trustflow](../images/easy_trustflow.jpg)

这些sidecar的能力如下：

1. Ra Proxy：能够生成远程认证报告，兼容各类TEE硬件并对外提供远程认证报告查询服务。

2. Data Capsule Proxy：能够从CapsuleManager获取数据密钥（对称密钥）和通信密钥（TLS证书、私钥），并给应用提供透明的数据读写接口（比如提供OSS透明加解密读写接口）。由于从CapsuleManager获取密钥的步骤中需要提供远程认证报告，所以这个Proxy本身也具备生成远程认证报告的能力。接口详情请见[Data Capsule Proxy接口](https://github.com/secretflow/secure-data-capsule-apis/blob/main/secretflowapis/v2/sdc/data_capsule_proxy/data_capsule_proxy.proto)。

3. Trustflow Envoy：应用流量的代理，基于Envoy开发。它对应用的出入流量进行统一处理。支持明文流量路由、密文流量透明加解密，可以按需配置。

另外，CapsuleManager现在还负责可信App通信组的管理。我们认为通信数据可以视作输入数据的衍生，所以通信密钥的管理应该依附于数据密钥的管理。两个不同的可信App如果同时具备同一个数据密钥的访问权限，那么它们在任务中就是被允许相互通信的，它们通信的TLS私钥、证书将由CapsuleManager鉴权后颁发。

我们通过几个[典型场景](../quick_start/easy_trustflow_examples.ipynb)来说明Easy-Trustflow的使用方法。